In [1]:
import astropy.units as u
import numpy as np
import popsims
import matplotlib.pyplot as plt
import astropy.coordinates as astro_coord
import glob
import pandas as pd

import itertools
from scipy import stats

from popsims.plot_style import  plot_style
plot_style()

#from shapey import Box
#paths
path_plot ='../figures/'
path_data = '../data/stream/'
isochrone_path='../data/isochrones/'
path_pipeline= '../data/pipeline/'

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import matplotlib as mpl
from scipy.interpolate import UnivariateSpline
from popsims.plot_tools import plot_annotated_heatmap
from matplotlib import patches
from easyshapey import Box

In [2]:
#SOME CONSTANTS

dgrid=np.arange(0.5, 10, 0.2)
BEST_BW=0.8
stream_size=0.1

THEORITICAL_STREAM_WIDTHS={'10_20': 0.053, '30_40': 0.094, '50_60':1.53}
GAP_SIZES={'10_20': 1.4, '30_40': 1.8, '50_60': 1.8}

GAP_PERCENTILE=90

PATH_POLYNOMIALS = np.load('../data/polynomial_stream_paths.npy', allow_pickle=True).flatten()[0]

FMTS={'10_20': 'o', '30_40': '*', '50_60': '^'}

In [3]:
def make_box(center, xextent, yextent):
    b=Box()
    x_min, x_max =center[0]- xextent/2, center[0]+ xextent/2
    y_min, y_max =center[-1]- yextent/2, center[-1]+ yextent/2
    v1= (x_min, y_min)
    v2=(x_min, y_max)
    v4= (x_max, y_min)
    v3=(x_max,y_max)

    b.vertices=[v1, v2, v3, v4, v1]
    return b

In [4]:
def make_line_plot(ax, x, y, yerr, rgc, show_error=False, **kwargs):
    
    nans=np.logical_or.reduce([np.isnan(x), np.isnan(y), np.isnan(yerr)])
    #print (vls)
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))
    
    y1 = np.interp(dgrid, x, y) 
    dy1 = np.interp(dgrid, x, y-yerr)
    dy2 = np.interp(dgrid, x, y+yerr)
    
    ax.plot(dgrid, y1, **kwargs)
    
    if show_error:
        ax.fill_between(dgrid, dy1, dy2, alpha=0.1, color=colors[rgc])

In [5]:
def get_counts(df, dgrid, rgc, m):
    
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))
    
    bools=np.logical_and.reduce([(df.bw==BEST_BW).values,\
                                 (df.rgc==rgc).values,
                                 (df.mag_limit==m).values])
    
    #bckd=read_backgrounds_file(m, rgc)
    #bckd['distance']=10**((bckd.dmod.values/5)+1)
    
    #print (bools)
    df0=(df[bools]).reset_index(drop=True)
    #jkl
    res={'gap_dens':[],
         'stream_dens':[],
         'bck_dens':[],
         'gap_stream_ratio':[],
         'stream_bck_ratio':[],
         'gap_bck_ratio':[],
         'st_error_percent':[],
         'gp_error_percent':[],
         'bck_error_percent':[],
         'gp_snr':[],
         'str_snr':[],
         'gap_spread':[],
         'median_gaploc':[],
         'median_treamloc':[],
         'gp_minus_bck_str_minus_bck':[],
         'gp_minus_bck':[],
         'std_gaploc':[],
         'med_gaploc':[],
         'std_streamloc':[],
         'fr_out_gap':[],
         'fr_stars_out_gap':[],
         'std_gaploc_gapsize_ratio':[],
         'med_gaploc_gapsize_ratio':[],
         'gap_area_over_true_area':[],
         'range_gaploc_gapsize_ratio':[],
         'gp_minus_bck_bck_ratio':[],
         'gp_minus_st_st_ratio':[],
         'ds': dgrid}
    
    for idx in range(len(dgrid)-1):
        v=df0[np.logical_and(df0.distance/1e6>=dgrid[idx], df0.distance/1e6<dgrid[idx+1])]
        
        #vbck=bckd[np.logical_and(bckd.distance/1e6>=dgrid[idx], bckd.distance/1e6<dgrid[idx+1])]
        
        st, st_err=  np.nanmedian(v.stream_dens.values),  np.nanstd(v.stream_dens.values)
        gp, gp_err=  np.nanmedian(v.gap_dens.values),  np.nanstd(v.gap_dens.values)
        #bck, bck_err=  np.nanmedian(vbck.density.values), np.nanstd(vbck.density.values)
        bck, bck_err=np.nanmedian(v.bck_dens.values), np.nanstd(v.bck_dens.values)
        
        #densities
        res['stream_dens'].append([st, st_err])
        res['gap_dens'].append([gp, gp_err])
        res['bck_dens'].append([bck, bck_err])

        stream_bck_ratio=st/bck
        gap_bck_ratio= gp/bck
        gap_st_ratio= gp/st
        #errors
        stream_bck_ratio_er= ((st_err/st)**2+ (bck_err/bck)**2)**0.5
        gap_bck_ratio_er= ((gp_err/gp)**2+ (bck_err/bck)**2)**0.5
        gap_st_ratio_er= ((gp_err/gp)**2+ (st_err/st)**2)**0.5
        
        res['gap_stream_ratio'].append([gap_st_ratio,gap_st_ratio_er])
        res['gap_bck_ratio'].append([gap_bck_ratio,gap_bck_ratio_er])
        res['stream_bck_ratio'].append([stream_bck_ratio,stream_bck_ratio_er])
        
        res['gp_minus_bck_bck_ratio'].append([(gp-bck)/st, 0])
        res['gp_minus_st_st_ratio'].append([(gp-st)/st, 0])

        
    
        res['st_error_percent'].append([st_err/st, 0.])
        res['gp_error_percent'].append([gp_err/gp, 0])
        res['bck_error_percent'].append([bck_err/bck, 0])
        
        res['gp_minus_bck_str_minus_bck'].append([(gp-bck)/(st-bck), 0])
        res['gp_minus_bck'].append([gp-bck, 0])
        
        
        gap_area= np.nanmedian(v.gap_area.values)
        gap_area_err= np.nanstd(v.gap_area.values)
        
        true_gap_area= GAP_SIZES[rgc]*0.2#np.nanmedian(v.spread_stream_stars_y.values)
        res['gap_area_over_true_area'].append([gap_area/true_gap_area,  gap_area_err/true_gap_area])
        cs= v.gap_points.values
    
        csx=[]
        
        if len(cs)>0:
            csx=np.concatenate([x for x in cs if len(x)>0])
    
            diffs=np.abs(csx-rgcx)
            med= np.nanmedian(diffs)
            std=np.nanstd(diffs)
            rang= np.ptp(diffs)
            
            fr= len(diffs[np.abs(diffs>0.5*GAP_SIZES[rgc])])/len(diffs)
            
            res['fr_out_gap'].append([fr, 0])
            res['std_gaploc_gapsize_ratio'].append([std/GAP_SIZES[rgc], 0])
            res['med_gaploc_gapsize_ratio'].append([med/GAP_SIZES[rgc], 0])
            res['range_gaploc_gapsize_ratio'].append([rang//GAP_SIZES[rgc], 0])
            res['std_gaploc'].append([std, 0])
            res['med_gaploc'].append([med, 0])
        else:
            res['fr_out_gap'].append([np.nan, 0])
            res['std_gaploc_gapsize_ratio'].append([np.nan, 0])
            res['med_gaploc_gapsize_ratio'].append([np.nan, 0])
            res['range_gaploc_gapsize_ratio'].append([np.nan, 0])
            res['std_gaploc'].append([np.nan, 0])
            res['med_gaploc'].append([np.nan, 0])
            
        
        cs= v.gap_stars.values
        csx=[]
        if len(cs)>0:
            csx=np.concatenate([x for x in cs if len(x)>0])
    
            diffs=np.abs(csx-rgcx)
            med= np.nanmedian(diffs)
            std=np.nanstd(diffs)
            rang= np.ptp(diffs)
            
            fr= len(diffs[np.abs(diffs>0.5*GAP_SIZES[rgc])])/len(diffs)
            
            res['fr_stars_out_gap'].append([fr, 0])
        else:
            res['fr_stars_out_gap'].append([np.nan, 0])
            

    return res

In [6]:
def plot():
            if plot=='show_mask_data':
                ax.scatter(data_df.x, data_df.y,  s=ms, alpha=alpha, c='k')
                ax.scatter(data_df.x[stream_track_on_data], data_df.y[stream_track_on_data],  s=10, alpha=0.1, c='r', label='Stream')
                ax.scatter(data_df.x[gap_track_on_data], data_df.y[gap_track_on_data],  s=10, alpha=0.1, c='b', label='Gap')

            if plot=='show_mask_grid':
                ax.scatter(grid_df.x, grid_df.y,  s=ms, alpha=alpha, c='k')
                ax.scatter(grid_df.x[stream_track], grid_df.y[stream_track],  s=10, alpha=1, c='r', label='Stream')
                ax.scatter(grid_df.x[gap_track], grid_df.y[gap_track],  s=10, alpha=1, c='b', label='Gap')
                #ax.scatter(grid_df.x[stream_track], grid_df.y,  s=10, alpha=0.1, c='b', label='Bckgd')

            if plot=='show_gap_points':
                #ax.scatter(grid_df.x, grid_df.y,  s=ms, alpha=alpha, c='k')
                #ax.scatter(grid_df.x[stream_track], grid_df.y[stream_track],  s=10, alpha=1, c='r', label='Stream')
                ax.scatter(grid_data[:,0][gap_track],grid_data[:,1][gap_track],  s=ms, alpha=alpha, c='#01FF70', marker='*')
                #ax.scatter(grid_df.x[stream_track], grid_df.y,  s=10, alpha=0.1, c='b', label='Bckgd')

            if plot=='plot0':
                #verify by plotting
                ax.scatter(vld['data'][0], vld['data'][1],s=10, alpha=alpha, c='k')

            if plot=='plot1':

                p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['density'],  20, \
                                  cmap='Greys', alpha=.2, linestyle='--')
                ax.contourf(vld['meshgrid'][0], vld['meshgrid'][1],  vld['max_eigen'],  20, \
                                  cmap='magma_r', alpha=1)
                #ax.scatter(grid_data[:,0], grid_data[:,1], c=res['max_eigen'].flatten(),
                #           s=1, cmap='cubehelix')

            if plot=='plot2':

                ax.contourf(vld['meshgrid'][0], vld['meshgrid'][1],  vld['min_eigen'],  20, \
                                  cmap='cubehelix', alpha=1)
                p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['density'],  20, \
                                  cmap='Greys', alpha=.2, linestyle='--')

            if plot=='plot6':
                #ax.scatter(vld['data'][:,0], vld['data'][:,1],s=0.1, alpha=0.5, c='k')
                ax.contourf(vld['meshgrid'][0], vld['meshgrid'][1],  vld['max_eigen'],  20, \
                                  cmap='magma_r', alpha=alpha, vmin=0, vmax=1)
                p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['density'],  20, \
                                                cmap='Greys', alpha=1, vmin=0.0, vmax=0.2, linestyle='--')



            if plot=='plot4':
                #verify by plotting
                #ax.scatter(vld['data'][0], vld['data'][1],s=ms, alpha=alpha,  c='#FFFFFF')
                #ax.scatter(meshgrid[0][stream_mask], meshgrid[1][stream_mask], marker='*', s=15,c='#FF851B')
                ax.scatter(meshgrid[0][gap_mask], meshgrid[1][gap_mask], marker='*', s=15,  c='#0074D9')

                #p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['max_eigen'],  20, color='b', alpha=0.5)

                #plot stream track
                ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])+stream_size/2, c='#FF851B', linewidth=3)
                ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])-stream_size/2, c='#FF851B', linewidth=3)

                #plot gap track
                ax.vlines(x=grid_data[:,0][gap_track].min(), \
                           ymin=pol(grid_data[:,0][gap_track].min())-stream_size/2,
                           ymax=pol(grid_data[:,0][gap_track].min())+stream_size/2, color='#0074D9', linewidth=3)
                ax.vlines(x=grid_data[:,0][gap_track].max(), \
                           ymin=pol(grid_data[:,0][gap_track].max())-stream_size/2,
                           ymax=pol(grid_data[:,0][gap_track].max())+stream_size/2, color='#0074D9', linewidth=3)
                #limit
                ax.set(xlim=[np.min(vld['data'][0]), np.max(vld['data'][0])],\
                       ylim=[np.min(vld['data'][1]), np.max(vld['data'][1])])

            if plot=='plot3':
                #verify by plotting
                #ax.scatter(vld['data'][:,0], vld['data'][:,1], s=10, alpha=alpha, c='k')
                #p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['max_eigen'],  20, color='b', alpha=0.5)

                #plot stream track
                ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])+stream_size/2, color='#FF851B', linewidth=3, alpha=alpha)
                ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])-stream_size/2, c='#FF851B', linewidth=3, alpha=alpha)

                #plot gap track
                ax.vlines(x=grid_data[:,0][gap_track].min(), \
                           ymin=pol(grid_data[:,0][gap_track].min())-stream_size/2,
                           ymax=pol(grid_data[:,0][gap_track].min())+stream_size/2, color='#FF851B', linewidth=3)
                ax.vlines(x=grid_data[:,0][gap_track].max(), \
                           ymin=pol(grid_data[:,0][gap_track].max())-stream_size/2,
                           ymax=pol(grid_data[:,0][gap_track].max())+stream_size/2, color='#FF851B', linewidth=3)
                #limit
                ax.set(xlim=[np.min(vld['data'][0]), np.max(vld['data'][0])],\
                       ylim=[np.min(vld['data'][1]), np.max(vld['data'][1])])

            if plot=='plot7':
                p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['density'],  20, \
                                  cmap='Greys', alpha=alpha*3, linestyle='--')
                #plot stream track
                ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])+stream_size/2, color='#FF851B', linewidth=3, alpha=alpha)
                ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])-stream_size/2, c='#FF851B', linewidth=3, alpha=alpha)

                #plot gap track
                ax.scatter(np.nanmedian(grid_data[:,0][gap_track]), \
                           np.nanmax(vld['meshgrid'][1])-0.5, color='k', marker='$\u2193$', s=100)
                #ax.vlines(x=grid_data[:,0][gap_track].min(), \
                #           ymin=pol(grid_data[:,0][gap_track].min())-stream_size/2,
                #           ymax=pol(grid_data[:,0][gap_track].min())+stream_size/2, color='#FF851B', linewidth=3)
                #ax.vlines(x=grid_data[:,0][gap_track].max(), \
                #           ymin=pol(grid_data[:,0][gap_track].max())-stream_size/2,
                #           ymax=pol(grid_data[:,0][gap_track].max())+stream_size/2, color='#FF851B', linewidth=3)
                #limit
                ax.set(xlim=[np.min(vld['data'][0]), np.max(vld['data'][0])],\
                       ylim=[np.min(vld['data'][1]), np.max(vld['data'][1])])



            if plot=='plot5':
                #verify by plotting
                #ax.scatter(vld['data'][0], vld['data'][1], s=10, alpha=0.2, c='k')
                p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['max_eigen'],  20, cmap='magma_r', alpha=0.5)

                #plot stream track
                #ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])+stream_size/2, c='#0074D9', linewidth=3)
                #ax.plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])-stream_size/2, c='#0074D9', linewidth=3)

                #plot gap track
                #ax.vlines(x=grid_data[:,0][gap_track].min(), \
                #           ymin=pol(grid_data[:,0][gap_track].min())-stream_size/2,
                #           ymax=pol(grid_data[:,0][gap_track].min())+stream_size/2, color='#0074D9', linewidth=3)
                #ax.vlines(x=grid_data[:,0][gap_track].max(), \
                #           ymin=pol(grid_data[:,0][gap_track].max())-stream_size/2,
                #           ymax=pol(grid_data[:,0][gap_track].max())+stream_size/2, color='#0074D9', linewidth=3)
                #limit
                ax.set(xlim=[np.min(vld['data'][0]), np.max(vld['data'][0])],\
                       ylim=[np.min(vld['data'][1]), np.max(vld['data'][1])])


            if plot=='density':
                p= ax.contour(vld['meshgrid'][0], vld['meshgrid'][1],  vld['density'],  20, \
                                  cmap='Greys', alpha=alpha*3, linestyle='--')
                p= ax.contourf(vld['meshgrid'][0], vld['meshgrid'][1],  vld['density'],  20, \
                                  cmap='Greys', alpha=alpha*3, linestyle='--')
            
            return 


In [7]:
def plot_stream_center(ax, rgc, mag_limit, plot_percentiles=False ):
    
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))
    
    df_small= (df[np.logical_and.reduce([df.bw==BEST_BW, df.rgc==rgc, df.mag_limit==mag_limit])]).reset_index(drop=True)

    for g in df_small.groupby('dmod'):

        #compute median and standard devidation

        #only pick 50 iterataions

        gpnts=g[1].gap_points.values
        #print (len(gpnts))

        #for i in range(0, 4): << will change
        #print (len(gpnts))
        for i in  range(0, len(gpnts)):
            med= np.nanmedian(np.abs(gpnts[i]-rgcx))
            std= np.nanstd(gpnts[i])
            x=np.nanmedian(g[1].distance/1e6)
            p=ax.errorbar(x, med, fmt=FMTS[rgc], c=colors[rgc], alpha=0.3, ms=5)

In [8]:
#functions
def show_gap_detection(vld,  GAP_SIZE, plot='None', ax=None, alpha=0.1, ms=0.1):
    meshgrid= vld['meshgrid']
    data=vld['data']
    grid_data = np.hstack([ xi.ravel().reshape(-1,1) for xi in meshgrid])

    gap_mask = vld['max_eigen'] > np.percentile(vld['max_eigen'].flatten(),GAP_PERCENTILE ) 
    data_df=pd.DataFrame()
    data_df['x']=data[0]
    data_df['y']=data[1]

    grid_df=pd.DataFrame()
    grid_df['x']= grid_data[:,0]
    grid_df['y']= grid_data[:,1]

    if np.isnan( vld['max_eigen']).all(): return {}

    pol=PATH_POLYNOMIALS[rgc]

    rgcx=np.nanmedian(np.array(rgc.split('_')).astype(float))

    #define stream track and gap track
    stream_track=np.abs(grid_data[:,1] - pol(grid_data[:,0]))< stream_size/2
    stream_track_on_data=np.logical_and.reduce([(np.abs(pol( data_df.x)-data_df.y)<stream_size/2).values,
                                                data_df.x > grid_df.x.min(),
                                                data_df.x < grid_df.x.max(),
                                                data_df.y > grid_df.y.min(),
                                                data_df.y < grid_df.y.max()])
                                                
    nstars_stream=len( data_df[stream_track_on_data])
    stream_area= (len(grid_df[stream_track])/len(grid_df))*(5*2)
    
    gap_center=(np.nanmedian(meshgrid[0][gap_mask]), rgcx )
    #choose a random place to ak
    b3= make_box( (gap_center[0], rgcx), GAP_SIZE, 0.2) #gap size
    gap_stars=b3.select(np.array(vld['data']))[0]
    gap_points=b3.select(np.array(grid_data).T)[0]
    gap_area= GAP_SIZE*0.2

    #bckd
    b1= make_box( (rgcx, rgcx+np.random.uniform(0.5, 1)), 5,0.5) #bckground
    nstars_bck=float(len(b1.select(np.array(vld['data']))[0]))
    nstars_gap=float(len(gap_stars))
    bck_area= 5*0.5

    #compute relevant statistics
    gap_offset=5
    meshgrid=[np.array(meshgrid[0]), np.array(meshgrid[1])]

    gap_offset=np.nanmedian(np.abs(meshgrid[0][gap_mask]-np.nanmedian(meshgrid[0])))

    stats={'spread_gap_stars_x': np.ptp(meshgrid[0][gap_mask]),
        'spread_gap_stars_y':np.nanstd(meshgrid[1][gap_mask]),
           'nstars_gap':  nstars_gap,
           'nstars_stream':  nstars_stream,
           'nstars_bck': nstars_bck,
           'stream_area': stream_area,
           'gap_area': gap_area,
           'bck_area': bck_area,
           'stream_dens':  np.log10(np.divide(nstars_stream, stream_area)),
           'gap_dens': np.log10(np.divide(nstars_gap, gap_area)),
           'bck_dens':np.log10(np.divide(nstars_bck, bck_area)),
           'offset_center':gap_offset,
           'gap_center':  gap_center[0],
           'gap_points':gap_points,
           'stream_center': np.nanmedian(grid_data[:,1][stream_track]),
           'stream_points':grid_data[:,1][stream_track],
           'stream_stars':data[1][stream_track_on_data],
           'gap_stars':gap_stars}
    return stats


In [9]:
def get_pipeline_results_full(dmod, mag_limit, rgc, bwu=BEST_BW):
    fls=glob.glob(path_pipeline+'/pipeline_rgc{}_mhalo5.00e+06_maglimit{:.2f}_run*.npy'.format(rgc, mag_limit))
    #print (fls
    res=[]
    for f in fls:
        c=np.load(f, allow_pickle=True)
        for vls in c:
            for k in vls.keys():
                bw=(k.split('dmod_galaxy')[0]).split('bw')[-1]
                dmodx=float(k.split('dmod_galaxy')[-1])
                #print (bw)
                if (dmod ==dmodx) and (float(bw)==bwu):
                    res.append(vls[k])
                else:
                    continue
    return res

In [ ]:
df = pd.DataFrame()
for f in tqdm(glob.glob(path_pipeline+'*.npy')):
    vlsx=np.load(f, allow_pickle=True)
    for vls in vlsx:
        for k in vls.keys():
            #for i in range(0, 10):
            rgc=f.split('rgc')[-1].split('_mhalo')[0]
            r=pd.Series(show_gap_detection(vls[k],  GAP_SIZES[rgc]))#, ax=ax, plot='plot3'))

            r['rgc']=rgc
            r['run']=f.split('_')[-1].split('.npy')[0]
            r['mag_limit']=float(f.split('_')[-2].split('maglimit')[1])
            r['bw']=float((k.split('dmod_galaxy')[0]).split('bw')[-1])
            r['dmod']=float(k.split('dmod_galaxy')[-1])
            #r['trial']=i


            df=df.append(r, ignore_index=True )

 40%|████████████████████████████████▊                                                 | 14/35 [1:10:02<2:42:41, 464.81s/it]

In [ ]:
bws=np.unique(df.bw)
bws

In [ ]:
np.random.uniform(0.5, 1)

In [ ]:
fig, ax=plt.subplots()
mask=np.logical_and(df.bw==0.8, df.rgc=='10_20')
plt.scatter(df.dmod[mask], df.offset_center[mask], alpha=0.5)
#plt.scatter(df.dmod[mask], (df.stream_dens-df.gap_dens)[mask], alpha=1)
#ax.set(yscale='log')

In [ ]:
df[np.logical_and.reduce([df.dmod==24.16, df.bw==0.8, df['rgc']=='30_40'])]

In [ ]:
len(np.unique(df.dmod)), np.unique(df.bw)

In [ ]:
df.rgc

In [ ]:
dmods2=[ 24.52,  25.  , 25.88, 26.51, 26.99,  28.01]

In [ ]:
fig, ax=plt.subplots(nrows=3, ncols=2, figsize=(14, 10.9), sharex=True, sharey=True)
rgc='30_40'
rgcx=35
xlim=[rgcx-3.5, rgcx+3.5]
ylim=[rgcx-2, rgcx+2]

bw=0.9

axs= np.concatenate(ax)
for idx in range(len(axs)):
    a=axs[idx]
    dmod=dmods2[idx]
    #print (dmod)
    #print (rgc)
    #print (bw)
    
    res=get_pipeline_results_full(dmod, 28.69, rgc, bwu=bw)[0]
    #print (res)
    #_=show_gap_detection(res,  plot='plot3', ax=a)
    dmod=float(dmod)
    #print(dmod)
    a.scatter(res['data'][0], res['data'][1], s=0.01, c='k')
    #a.hist2d(res['data'][0], res['data'][1], bins=int(28*1.5), range=[xlim, ylim], cmap='Greys')
    a.minorticks_on()
    #a.axis('equal')
    
    d_galaxy=(1e-3*(10**((dmod/5)+1)))*u.kpc
    kpc_conversion = np.pi * d_galaxy / 180.
    roman_fov= 0.52*u.degree*(kpc_conversion /u.degree)
    scaled_arcmin= 5*u.arcmin.to(u.degree)*(kpc_conversion /u.degree).value
    
    a.hlines(y=rgcx-1.2, xmin= rgcx-3, xmax=rgcx-3+scaled_arcmin, \
             linewidth=5,  color='#FF851B')
    
    rect = patches.Rectangle( (rgcx-3, rgcx-1.1), \
                               1.5, 0.24, linewidth=1, edgecolor='#FFFFFF', facecolor='#FFFFFF')
    a.add_patch(rect)
    a.text(rgcx-3, rgcx-1.1, '5 Arcmin',\
           color='#FF851B', alpha=1, fontsize=14, weight='bold')
    
    a.text(rgcx-0.7,  rgcx+0.5, u'\u007d', fontsize=70, ha='left', va='bottom', rotation=90,  color='#FF851B')
    #a.scatter(34.5,  35.0+0.3, color='#0074D9', marker='$\u2193$', s=600)
    
    

    a.set(title='{:.2f} Mpc'.format(1e-6*(10**((dmod/5)+1))), xlabel='x (kpc)', ylabel='y (kpc)', 
    xlim=xlim, ylim=ylim)

plt.tight_layout()
plt.savefig('../figures/gaps_visual_detections{}_nometcut.jpeg'.format(rgc))

In [ ]:
res['data'][1].min()-res['data'][1].max()

In [ ]:
np.round(100/len(res['data'][0]), 1)

In [ ]:
def plot_grid_lines(x, y, ax):
    # Plot the vertical grid lines
    for i in range(len(x)):
        ax.plot([x[i], x[i]], [y[0], y[-1]], 'k-', alpha=0.1,  linewidth=0.05)

    # Plot the horizontal grid lines
    for i in range(len(y)):
        ax.plot([x[0], x[-1]], [y[i], y[i]], 'k-', alpha=0.1,  linewidth=0.05)



In [ ]:
fig, ax=plt.subplots(nrows=2, ncols=2, figsize= (7*2, 4*1.9))
dmod= 24.52
rgc='30_40'
bw=0.8
res=get_pipeline_results_full( dmod, 28.69, rgc, bwu=bw)[-1]
#resx= np.load('../data/test.npy', allow_pickle=True).flatten()[0]
#res=resx[list(resx.keys())[0]]
#print (resx)
dt=res['data']
vld=res
gap_mask = vld['max_eigen'] > np.percentile(vld['max_eigen'].flatten(),GAP_PERCENTILE ) 

grid_data=res['grid_data']
for a in np.concatenate(ax):
    a.scatter(dt[0], dt[1], s=.3, alpha=0.1, c='k')
    a.set(xlim=[dt[0].min(), dt[0].max()], xlabel='x (kpc)', ylabel='y (kpc)')
    a.minorticks_on()
    plot_grid_lines(grid_data[:,0], grid_data[:,1], a)
    
ax[0][0].scatter(dt[0], dt[1], s=.1, alpha=0.5, c='k')
ax[-1][-1].scatter(dt[0], dt[1], s=.1, alpha=0.5, c='k')

ax[0][1].contour(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                              cmap='cubehelix', alpha=0.5, linestyle='--')
ax[0][1].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                              cmap='cubehelix', alpha=0.5, linestyle='--')


ax[1][0].contour(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                              cmap='YlOrBr', alpha=0.5, linestyle='--')
ax[1][0].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                              cmap='YlOrBr', alpha=0.5, linestyle='--')



#ax[1][1].contour(res['meshgrid'][0], res['meshgrid'][1],  res['min_eigen'],  20, \
#                              cmap='YlOrBr', alpha=0.5, linestyle='--')
#ax[1][1].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['min_eigen'],  20, \
#                              cmap='YlOrBr', alpha=0.5, linestyle='--')

pol=PATH_POLYNOMIALS[rgc]
stream_size=0.2
#ax[1][-1].plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])+stream_size/2, color='#FF851B', linewidth=1)
#ax[1][-1].plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])-stream_size/2, c='#FF851B', linewidth=1)



ax[1][-1].scatter(vld['meshgrid'][0][gap_mask], vld['meshgrid'][1][gap_mask], marker='*', color='#0074D9')

ax[0][1].scatter(dt[0], dt[1], s=.1, alpha=0.5, c='k')
ax[1][0].scatter(dt[0], dt[1], s=.1, alpha=0.5, c='k')

xlim=[35-2.5, 35+2.5]
ylim=[35-1, 35+1]
ax[-1][-1].set(xlim=xlim, ylim=ylim)
ax[-1][0].set(xlim=xlim, ylim=ylim)
ax[0][1].set(xlim=xlim, ylim=ylim)
cmap = mpl.cm.magma_r
norm = mpl.colors.Normalize(vmin=0, vmax=1)
cax = fig.add_axes([-0.05, 0.15, 0.01, 0.3])
cax.yaxis.set_ticks_position('left')
cax.yaxis.set_label_position('left')
cax.yaxis.tick_right()
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
            cax=cax, orientation='vertical', label=r'Max Eigenvalue of $\Pi H \Pi$')


cmap = mpl.cm.cubehelix
norm = mpl.colors.Normalize(vmin=0, vmax=1)
cax = fig.add_axes([0.985, 0.625, 0.01, 0.3])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cax, orientation='vertical', label=r'Normalized Density')
plt.tight_layout()
#plt.savefig('../figures/including_more_data_example.jpeg')
# Add the text annotation with an arrow
ax[1][-1].annotate('Gap indicator points', xy=(35, 35.5), xytext=(33.5, 35.75),
            arrowprops=dict(arrowstyle='->', color='b'))



plt.savefig('../figures/stream_cutout.jpeg', bbox_inches='tight')


In [ ]:
colors={'10_20': '#0074D9',
       '30_40': '#111111', 
       '50_60': '#FF851B' }

In [ ]:
df['distance']=10**((df.dmod.values/5)+1)

In [ ]:
fig, ax=plt.subplots(figsize=(12, 12), ncols=2, nrows=4, sharey='row',  gridspec_kw={'height_ratios': [5, 1, 1, 1]})

for rgc in ['10_20', '30_40', '50_60']:
    
    vls=get_counts(df, dgrid,rgc, 27.15)
  
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))

    make_line_plot( ax[0][0], dgrid[:-1], np.vstack(vls['stream_dens'])[:,0], np.vstack(vls['stream_dens'])[:,1],\
                   rgc, color=colors[rgc], show_error=True, linestyle='-', label=r'Stream, R$_{\rm GC}$'+'={} kpc'.format(rgcx))
    
    make_line_plot( ax[0][0], dgrid[:-1], np.vstack(vls['gap_dens'])[:,0], np.vstack(vls['gap_dens'])[:,1],\
                   rgc, color=colors[rgc], show_error=True, linestyle='--', label=r'Gap, R$_{\rm GC}$'+'={} kpc'.format(rgcx), linewidth=3)
    
    #make_line_plot( ax[0][0], dgrid[:-1], np.vstack(vls['bck_dens'])[:,0], np.vstack(vls['bck_dens'])[:,1],\
    #               rgc, color=colors[rgc], linestyle=':', label=r'Background,R$_{\rm GC}$'+'={} kpc'.format(rgcx))
    
  
    #make_line_plot( ax[1][0], dgrid[:-1], np.vstack(vls['gp_error_percent'])[:,0], np.vstack(vls['gp_minus_bck'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='--', label='Rgc={} Kpc'.format(rgcx))
    #make_line_plot( ax[1][0], dgrid[:-1], np.vstack(vls['st_error_percent'])[:,0], np.vstack(vls['gp_minus_bck'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='-', label='Rgc={} Kpc'.format(rgcx))
    #make_line_plot( ax[1][0], dgrid[:-1], np.vstack(vls['gap_area_over_true_area'])[:,0], np.vstack(vls['gap_area_over_true_area'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='-', label='Rgc={} Kpc'.format(rgcx))

  
    vls=get_counts(df, dgrid,rgc, 28.69)
  
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))

    make_line_plot( ax[0][1], dgrid[:-1], np.vstack(vls['stream_dens'])[:,0], np.vstack(vls['stream_dens'])[:,1],\
                   rgc, color=colors[rgc], show_error=True, linestyle='-', label=r'Stream, R$_{\rm GC}$'+'={} kpc'.format(rgcx))

    make_line_plot( ax[0][1], dgrid[:-1], np.vstack(vls['gap_dens'])[:,0], np.vstack(vls['gap_dens'])[:,1],\
                   rgc, color=colors[rgc], show_error=True, linestyle='--',label=r'Gap, R$_{\rm GC}$'+'={} kpc'.format(rgcx), linewidth=3)
    
    #make_line_plot( ax[0][1], dgrid[:-1], np.vstack(vls['bck_dens'])[:,0], np.vstack(vls['bck_dens'])[:,1],\
    #               rgc, color=colors[rgc], linestyle=':', label=r'Background, R$_{\rm GC}$'+'={} kpc'.format(rgcx))
    
    
    #make_line_plot( ax[1][1], dgrid[:-1], np.vstack(vls['gp_error_percent'])[:,0], np.vstack(vls['gp_minus_bck'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='--', label='Rgc={} Kpc'.format(rgcx))
    #make_line_plot( ax[1][1], dgrid[:-1], np.vstack(vls['st_error_percent'])[:,0], np.vstack(vls['gp_minus_bck'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='-', label='Stream,Rgc={} Kpc'.format(rgcx))
    #make_line_plot( ax[1][1], dgrid[:-1], np.vstack(vls['gap_area_over_true_area'])[:,0], np.vstack(vls['gap_area_over_true_area'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='-', label='Rgc={} Kpc'.format(rgcx))
    
    
    #fig, ax=plt.subplots(ncols=10, nrows=14, figsize=(20, 20))
    
   
#a.set( xlabel='Distance Modulus', ylabel='Bandwidth (kpc)', ylim=[0.05, 1.65], xlim=[23.7, 29.35])
rgcs=['10_20', '30_40', '50_60']

for idx in range(0,3):
    rgc=rgcs[idx]
    plot_stream_center(ax[1+idx][1], rgc, 28.69 )
    plot_stream_center(ax[1+idx][0], rgc, 27.15)
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))
    a=ax[idx+1]
    a[0].set_title(r'1000 s, R$_{\rm GC}$'+' = {:.0f} kpc'.format(rgcx),fontsize=14)
    a[1].set_title(r'1 hour, R$_{\rm GC}$'+' = {:.0f} kpc'.format(rgcx),fontsize=14)
    #a[0].set(ylim=[0.0, 1.])

for a in np.concatenate(ax):
    a.minorticks_on()
    a.set(xscale='log',  xticks=[1, 2, 3, 4, 5, 7, 10],xticklabels=[1, 2, 3, 4, 5, 7, 10], xlabel='Distance (Mpc)')

for a in ax[0]:
    #a.legend(fontsize=10)
    #a.set(yticks=10**np.array([-1., 0, 1., 2, 2., 3, 4]))
    a.yaxis.set_tick_params(labelbottom=True)
    a.set(ylabel='Log Stellar Density (kpc$^{-2}$)')
    

ax[0][1].set_title('1 hour'.format(rgcx),fontsize=14)
ax[0][0].set_title('1000 s'.format(rgcx),fontsize=14)


for a in np.concatenate([ax[1], ax[2], ax[3]]):
    #a.set(yscale='log')
    #a.legend(fontsize=10)
    #a.axhline(0.2, linestyle='-', color='k', alpha=0.5, linewidth=3)
    #a.axhline(1, linestyle='-', color='k', alpha=0.5, linewidth=3)
    #a.axhline(0.25, linestyle='-', color='k', alpha=0.5, linewidth=3)
    
    # a.axhline(GAP_SIZE, linestyle='--', color='k', alpha=0.5, linewidth=3)
    #a.axhline(GAP_SIZE/2, linestyle='--', color='k', alpha=0.5, linewidth=3)
    #a.text(1, GAP_SIZE+0.05, 'Gap Size', fontsize=14)
    #a.text(1, GAP_SIZE/2+0.05, 'Gap Size/2', fontsize=14)
    

    plt.setp(a.get_xticklabels(), visible=True)
    
    a.yaxis.set_tick_params(labelbottom=True)
    
    _= a.set(ylabel=r'$\Delta$ (kpc)'.format(rgcx))#, ylim=[-0.5, 2.5])



#ax[1][0].set(ylim=[-2, 2])
#ax[1][1].set(ylim=[-2, 2])
plt.tight_layout()

plt.savefig('../figures/gap_density_errors_nometcut.pdf')

In [ ]:
df['rgcx']=df.rgc.apply(lambda x: np.nanmedian(np.array(x.split('_')).astype(float)))

In [ ]:
df['deltasigma']=df.offset_center+df.spread_gap_stars_x

In [ ]:
df0=df[df.rgc=='30_40']

In [ ]:
#mpl.cm.ScalarMappable?

In [ ]:
fig, ax=plt.subplots(ncols=2, figsize=(14, 5))

cmap = mpl.cm.Blues
norm1 =mpl.colors.LogNorm()
norm2 =mpl.colors.LogNorm()

plot_annotated_heatmap(ax[0], df0, 15, ['dmod', 'bw', 'spread_gap_stars_x'], norm=norm1,  cmap=cmap,annotate=False, alpha=1)
plot_annotated_heatmap(ax[1], df0, 15, ['dmod', 'bw', 'offset_center'],  norm=norm2, cmap=cmap,annotate=False, alpha=1)
#plot_annotated_heatmap(ax[-1], df0, 15, ['dmod', 'bw', 'deltasigma'],  norm=norm2, cmap=cmap,annotate=False, alpha=0.5)

#ax[0].scatter(df0.dmod, df0.bw, c=df0.spread_gap_stars_x, s=150, marker='*', norm=norm1, cmap=cmap, alpha=0.5)
#c=ax[1].scatter(df0.dmod, df0.bw, c=df0.offset_center, s=150, marker='*', norm=norm1, cmap=cmap, alpha=0.5)



xticks=[0.6, 1, 2, 3, 4]
cbar=plt.colorbar(mpl.cm.ScalarMappable(norm=norm1, cmap=cmap), ax=ax[0], orientation='vertical', label=r'Spread of Gap Points ($\mathcal{S}_g$, Kpc)')
cbar.ax.set_yticks([])
cbar.ax.set_yticklabels([])
cbar.ax.set(yticks=xticks, yticklabels=xticks)


xticks=[0.1,0.2,  0.5, 1, 2]
# Add colorbar, make sure to specify tick locations to match desired ticklabels
#cbar = fig.colorbar(cax, ticks=[-1, 0, 1]) #setting which ticks you actually want
#cbar.ax.set_yticklabels(['< -1', '0', '> 1'])  #satting the labels for those ticks
cbar=plt.colorbar(mpl.cm.ScalarMappable(norm=norm2, cmap=cmap), ax=ax[1], orientation='vertical', label=r'Gap Location from Center ($\Delta$, Kpc)')
cbar.ax.set(yticks=xticks, yticklabels=xticks)

def forward(x):
    return (10**((x/5)+1))/1e6

def inverse(x):
    return  5*np.log10(x*1000/(10))

for a in ax:
    secax = a.secondary_xaxis('top', functions=(forward, inverse))
    secax.set(xlabel='Distance (Mpc)')
    
    a.set( xlabel='Distance Modulus', yticks=[0.5, 1, 1.5, 2.],
          ylabel='Bandwidth (kpc)', ylim=[0.1, 1.85], xlim=[23.7, 29.7])


plt.tight_layout()


#actually make 3 stacks, get rid of first panel to emphasize that best bw doesn't depend on gap size
plt.savefig('../figures/gaps_spread_all_bws_nometcut.pdf', bbox_inches='tight')


In [ ]:
dmods=np.unique(df.dmod.values[np.logical_and(df.distance/1e6 <7, df.distance/1e6 > 0.5) ])
dmods=(dmods[::3])[:25]
len(dmods)
rgc='10_20'
rgcx=15

fig, ax=plt.subplots(ncols=3, nrows=int(len(dmods)/3), sharey=True, sharex=True, figsize=(14, 2*int(len(dmods)/3)))

axs= np.concatenate(ax)
for idx in range(len(axs)):
    a=axs[idx]
    dmod=dmods[idx]
    
    resx=np.random.choice(get_pipeline_results_full(dmod, 27.15, rgc, bwu=0.8), 10)
    meds=[]
    for res in resx:
        plotres=show_gap_detection(res, GAP_SIZES[rgc])
        mask= res['max_eigen']>  np.percentile(res['max_eigen'].flatten(), GAP_PERCENTILE) 
        Z_masked = np.ma.masked_array(res['max_eigen'], mask)
        p= a.contour(res['meshgrid'][0], res['meshgrid'][1],  Z_masked,  20, \
                                  cmap='magma_r', alpha=.1, linestyle='--')
        meds.append(np.nanmedian(plotres['gap_points']))
        
        dt=res['data']
        a.scatter(dt[0], dt[1], s=0.1, alpha=0.01, c='k')

    for m in meds:
        a.scatter(m, 33, color='k', marker='$\u2191$', s=100, zorder=10)


    
    rect = patches.Rectangle( (rgcx,rgcx+1.5), \
                               2, 0.3, linewidth=1, edgecolor='#FFFFFF', facecolor='#FFFFFF')
    #a.add_patch(rect)
    title='{:.1f} Mpc'.format(1e-6*(10**((dmod/5)+1)))
    a.text(rgcx, rgcx+1.5, title,\
           color='b', alpha=1, fontsize=14, weight='bold')
    xlim=[rgcx-2.7, rgcx+2.7]
    ylim=[rgcx-2.5, rgcx+2.5]
    a.set(xlabel='x (kpc)', ylabel='y (kpc)', xlim=xlim, ylim=ylim)
    
    
    del resx
#ax.scatter(res['meshgrid'][0], res['meshgrid'][1], marker='+', alpha=0.1)
plt.tight_layout()
plt.savefig('../figures/visual_detections_rgc35_nometcut.jpeg')


In [ ]:
DRTFYGHUJIK

In [ ]:
#plt.style.use('dark_background') #for posters

In [ ]:
resxs=np.load('../data/appendix_intact.npy', allow_pickle=True).flatten()


for idx in range(len(resxs)):
    resx=resxs[idx]
    ks= [k for k in resx.keys()]
    res=resx[ks[0]]
    print (idx, ks[0])

    #_= show_gap_detection(res,  GAP_SIZES[rgc], plot='density', ax=ax[idx][0], alpha=0.3)
    dt=res['data']
    fig, ax=plt.subplots(ncols=4, figsize= (16, 3))
    
    ax[0].scatter(dt[0], dt[1], s=0.1, alpha=0.5,c='k')

    ax[1].contour(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                                  cmap='cubehelix', alpha=0.5, linestyle='--')
    ax[1].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                                  cmap='cubehelix', alpha=0.5, linestyle='--')


    ax[2].contour(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                                  cmap='YlOrBr', alpha=0.5, linestyle='--')
    ax[2].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                                  cmap='YlOrBr', alpha=0.5, linestyle='--')

    bw=(ks[0].split('dmod')[0]).split('bw')[-1]
    #a.set(title='Bandwidth={} kpc'.format(bw))
    #ax[idx][1].set(title='Bandwidth={} kpc'.format(bw))
    
    pol=PATH_POLYNOMIALS[rgc]
    
    grid_data=res['grid_data']
    
    dt=res['data']
    vld=res
    gap_mask = vld['max_eigen'] > np.percentile(vld['max_eigen'].flatten(),GAP_PERCENTILE ) 

    ax[-1].scatter(vld['meshgrid'][0][gap_mask], vld['meshgrid'][1][gap_mask], marker='*', color='#0074D9')
    ax[-1].scatter(dt[0], dt[1], s=.1, alpha=0.5, c='k')
    
    
    cmap = mpl.cm.YlOrBr
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    cax = fig.add_axes([0.566, -0.05, 0.165, 0.05])
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                 cax=cax, orientation='horizontal', label=r'Max Eigenvalue of $\Pi H \Pi$')

    cmap = mpl.cm.cubehelix
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    cax = fig.add_axes([0.32, -0.05, 0.165, 0.05])
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                 cax=cax, orientation='horizontal', label=r'Density')



    for a in ax:
        a.scatter(dt[0], dt[1], s=.1, alpha=0.1, c='k')
        #a.set(xlim=[grid_data[0:,].min(), grid_data[:,0].max()], xlabel='x (kpc)', ylabel='y (kpc)')
        #a.set(ylim=[grid_data[1:,].min(), grid_data[:,1].max()])
        a.set(xlabel='x (kpc)', ylabel='y (kpc)')#, xlim=[35-2.5, 35+2.5], ylim=[35-1, 35+1])
        a.minorticks_on()

    plt.tight_layout()
    plt.savefig('../figures/appendix_unpeturbed{}.jpeg'.format(int(idx)), bbox_inches='tight')

In [ ]:
resxs=np.load('../data/appendix_offcenter.npy', allow_pickle=True).flatten()


for idx in range(len(resxs)):
    resx=resxs[idx]
    ks= [k for k in resx.keys()]
    print (idx, ks[0])
    res=resx[ks[0]]

    #_= show_gap_detection(res,  GAP_SIZES[rgc], plot='density', ax=ax[idx][0], alpha=0.3)
    dt=res['data']
    fig, ax=plt.subplots(ncols=4, figsize= (16, 3))
    ax[0].scatter(dt[0], dt[1], s=0.1, alpha=0.5, c='k')

    ax[1].contour(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                                  cmap='cubehelix', alpha=0.5, linestyle='--')
    ax[1].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                                  cmap='cubehelix', alpha=0.5, linestyle='--')


    ax[2].contour(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                                  cmap='YlOrBr', alpha=0.5, linestyle='--')
    ax[2].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                                  cmap='YlOrBr', alpha=0.5, linestyle='--')

    bw=(ks[0].split('dmod')[0]).split('bw')[-1]
    #a.set(title='Bandwidth={} kpc'.format(bw))
    #ax[idx][1].set(title='Bandwidth={} kpc'.format(bw))
    
    pol=PATH_POLYNOMIALS[rgc]
    
    grid_data=res['grid_data']
    
    dt=res['data']
    vld=res
    gap_mask = vld['max_eigen'] > np.percentile(vld['max_eigen'].flatten(),GAP_PERCENTILE ) 


    ax[-1].scatter(vld['meshgrid'][0][gap_mask], vld['meshgrid'][1][gap_mask], marker='*', color='#0074D9')
    ax[-1].scatter(dt[0], dt[1], s=.1, alpha=0.5, c='k')
    
    
    cmap = mpl.cm.YlOrBr
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    cax = fig.add_axes([0.566, -0.05, 0.165, 0.05])
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                 cax=cax, orientation='horizontal', label=r'Max Eigenvalue of $\Pi H \Pi$')

    cmap = mpl.cm.cubehelix
    norm = mpl.colors.Normalize(vmin=0, vmax=1)
    cax = fig.add_axes([0.32, -0.05, 0.165, 0.05])
    fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
                 cax=cax, orientation='horizontal', label=r'Density')



    for a in ax:
        a.scatter(dt[0], dt[1], s=.1, alpha=0.1, c='k')
        #a.set(xlim=[grid_data[0:,].min(), grid_data[:,0].max()], xlabel='x (kpc)', ylabel='y (kpc)')
        #a.set(ylim=[grid_data[1:,].min(), grid_data[:,1].max()])
        a.set(xlabel='x (kpc)', ylabel='y (kpc)')
        a.minorticks_on()

    plt.tight_layout()
    plt.savefig('../figures/appendix_offcenter{}.jpeg'.format(int(idx)),  bbox_inches='tight')

In [ ]:
#metrics based on densities
fig, ax=plt.subplots(figsize=(12, 6))#, height_ratios=[1.,  0.5])

for rgc in ['10_20', '30_40', '50_60']:
    
    vls=get_counts(df, dgrid,rgc, 28.69)
  
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))
    
    #make_line_plot( ax[0], dgrid[:-1], np.vstack(vls['gap_bck_ratio'])[:,0], np.vstack(vls['gap_bck_ratio'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='--', label='Rgc={} Kpc'.format(rgcx))
    make_line_plot( ax, dgrid[:-1], np.vstack(vls['gap_bck_ratio'])[:,0], np.vstack(vls['gap_bck_ratio'])[:,1],\
                   rgc, color=colors[rgc], linestyle='-', label='Rgc={} Kpc'.format(rgcx))
    
    vls=get_counts(df, dgrid,rgc, 27.15)
    
    #make_line_plot( ax[1], dgrid[:-1], np.vstack(vls['gap_bck_ratio'])[:,0], np.vstack(vls['gap_bck_ratio'])[:,1],\
    #               rgc, color=colors[rgc], linestyle='--', label='Rgc={} Kpc'.format(rgcx))
    make_line_plot( ax, dgrid[:-1], np.vstack(vls['gap_bck_ratio'])[:,0], np.vstack(vls['gap_bck_ratio'])[:,1],\
                   rgc, color=colors[rgc], linestyle='-.', label='Rgc={} Kpc'.format(rgcx))
    


In [ ]:
#metrics based on densities
fig, ax=plt.subplots(figsize=(12, 6))#, height_ratios=[1.,  0.5])

for rgc in ['10_20', '30_40', '50_60']:
    
    vls=get_counts(df, dgrid,rgc, 28.69)
  
    rgcx=int(np.nanmedian(np.array(rgc.split('_')).astype(float)))
    
    make_line_plot( ax, dgrid[:-1], np.vstack(vls['gap_stream_ratio'])[:,0], np.vstack(vls['gap_stream_ratio'])[:,1],\
                   rgc, color=colors[rgc], linestyle='-', label='Rgc={} Kpc'.format(rgcx))
    
    vls=get_counts(df, dgrid,rgc, 27.15)
    
    make_line_plot( ax, dgrid[:-1], np.vstack(vls['gap_stream_ratio'])[:,0], np.vstack(vls['gap_stream_ratio'])[:,1],\
                   rgc, color=colors[rgc], linestyle='-.', label='Rgc={} Kpc'.format(rgcx))

ax.set(xscale='log')


In [ ]:
#all bws only needs to be computed once, then multiple runs for 0.7 or whatevs
#alright looks like we are back to 5 x 2 because 2x1x2+5 = 9

In [ ]:
np.unique(df.dmod)

In [ ]:
2*2*2+5

In [ ]:
dmod= 29.06
1e-6*(10**((dmod/5)+1))

In [ ]:
fig, ax=plt.subplots(ncols=4, figsize= (16, 3))
dmod= 29.06
rgc='30_40'
bw=0.8
res=get_pipeline_results_full( dmod, 28.69, rgc, bwu=bw)[0]
#resx= np.load('../data/test.npy', allow_pickle=True).flatten()[0]
#res=resx[list(resx.keys())[0]]
#print (resx)
dt=res['data']
vld=res
gap_mask = vld['max_eigen'] > np.percentile(vld['max_eigen'].flatten(),GAP_PERCENTILE ) 

ax[0].scatter(dt[0], dt[1], s=1, alpha=0.5, c='k')

ax[1].contour(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                              cmap='cubehelix', alpha=0.5, linestyle='--')
ax[1].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
                              cmap='cubehelix', alpha=0.5, linestyle='--')


ax[2].contour(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                              cmap='YlOrBr', alpha=0.5, linestyle='--')
ax[2].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['max_eigen'],  20, \
                              cmap='YlOrBr', alpha=0.5, linestyle='--')



#ax[1][1].contour(res['meshgrid'][0], res['meshgrid'][1],  res['min_eigen'],  20, \
#                              cmap='YlOrBr', alpha=0.5, linestyle='--')
#ax[1][1].contourf(res['meshgrid'][0], res['meshgrid'][1],  res['min_eigen'],  20, \
#                              cmap='YlOrBr', alpha=0.5, linestyle='--')

pol=PATH_POLYNOMIALS[rgc]
#ax[-1].plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])+stream_size/2, color='#FF851B', linewidth=3)
#ax[-1].plot(vld['meshgrid'][0], pol(vld['meshgrid'][0])-stream_size/2, c='#FF851B', linewidth=3)

grid_data=res['grid_data']

ax[-1].scatter(vld['meshgrid'][0][gap_mask], vld['meshgrid'][1][gap_mask], marker='*', color='#0074D9')

for a in ax:
    a.scatter(dt[0], dt[1], s=1, alpha=0.1, c='k')
    #a.set(xlim=[grid_data[0:,].min(), grid_data[:,0].max()], xlabel='x (kpc)', ylabel='y (kpc)')
    #a.set(ylim=[grid_data[1:,].min(), grid_data[:,1].max()])
    a.set(xlabel='x (kpc)', ylabel='y (kpc)', xlim=[35-2.5, 35+2.5], ylim=[35-1, 35+1])
    a.minorticks_on()


cmap = mpl.cm.YlOrBr
norm = mpl.colors.Normalize(vmin=0, vmax=1)
cax = fig.add_axes([0.566, -0.05, 0.165, 0.05])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cax, orientation='horizontal', label=r'Max Eigenvalue of $\Pi H \Pi$')

cmap = mpl.cm.cubehelix
norm = mpl.colors.Normalize(vmin=0, vmax=1)
cax = fig.add_axes([0.32, -0.05, 0.165, 0.05])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=cax, orientation='horizontal', label=r'Density')

plt.tight_layout()
#plt.savefig('../figures/including_more_data_example.jpeg')
plt.savefig('../figures/appendix_failure2.jpeg', bbox_inches='tight')


In [ ]:
dmods=np.unique(df.dmod.values[np.logical_and(df.distance/1e6 <10, df.distance/1e6 > 0.7) ])
dmods=(dmods[::3])
len(dmods)
rgc='30_40'

fig, ax=plt.subplots(ncols=4, nrows=int(len(dmods)/4), figsize=(14, 2*int(len(dmods)/4)))

axs= np.concatenate(ax)
for idx in range(len(axs)):
    a=axs[idx]
    dmod=dmods[idx]
    
    resx=np.random.choice(get_pipeline_results_full(dmod, 27.15, rgc, bwu=0.8), 5)
    meds=[]
    for res in resx:
        plotres=show_gap_detection(res, GAP_SIZES[rgc])
        #p= a.contourf(res['meshgrid'][0], res['meshgrid'][1],  res['density'],  20, \
        #                          cmap='cubehelix', alpha=.1, linestyle='--')
        a.scatter(res['data'][0], res['data'][1], s=0.1, alpha=0.1, c='k')
        meds.append(np.nanmedian(plotres['gap_points']))

    for m in meds:
        a.scatter(m, 34.5, color='k', marker='$\u2191$', s=100, zorder=10)

    
    title='{:.2f} Mpc'.format(1e-6*(10**((dmod/5)+1)))
    
    rect = patches.Rectangle( (35,35.5), \
                               2, 0.3, linewidth=1, edgecolor='#FFFFFF', facecolor='#FFFFFF')
    a.add_patch(rect)
    a.text(35, 35.5, title,\
           color='b', alpha=1, fontsize=14, weight='bold')
    a.set(xlabel='x (kpc)', ylabel='y (kpc)')
    
    del resx
#ax.scatter(res['meshgrid'][0], res['meshgrid'][1], marker='+', alpha=0.1)
plt.tight_layout()
plt.savefig('../figures/visual_detections_rgc35_scatter.jpeg')


In [ ]:
meshgrid=res['meshgrid']

In [ ]:
#maybe don't need to show polynomial fit 

In [ ]:
 np.ptp(meshgrid[0][gap_mask])

In [ ]:
for d in 

##### 